Financial Large Language Models (FinLLMs) with FinGPT using LoRA (Low-Rank Adaptation) base model :: LLaMA-13B

FinGPT v3

In [1]:
! pip install torch
! pip install protobuf cpm_kernels torch>=2.0 gradio mdtex2html sentencepiece accelerate
! pip install huggingface_hub
! pip install peft
! pip install datasets
! pip install accelerate
! pip install bitsandbytes

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.7 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.6.4 which is incompatible.
spacy 3.4.1 requires pydantic!=1.8,!=1.8.1,<1.10.0,>=1.7.4, but you have pydantic 2.6.4 which is incompatible.
spacy 3.4.1 requires typer<0.5.0,>=0.3.0, but you have typer 0.10.0 which is incompatible.
confection 0.0.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.6.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 3.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 45.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 16.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
! pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-3yy_5ihm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-3yy_5ihm
  Resolved https://github.com/huggingface/transformers to commit 76a33a10923ccc1074917f6b6a1e719e626b7dc9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 42.7 MB/s eta 0:00:0000:0100:01
  Created wheel for transformers: filename=transformers-4.40.0.dev0-py3-none-any.whl size=8802691 sha256=5df43930ac0379c8335ffcdf9fe08cd942dcb6ee9b1bb909530c77fb8e4148ca
  Stored in directory: /tmp/pip-ephem-wheel-cache-bx43034m/wheels/14/a0/7b/8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190999ac2b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Su

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
import datasets
import sys

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    LlamaForCausalLM,
    LlamaTokenizerFast
)

from peft import (
    TaskType,
    LoraConfig,
    get_peft_model,
    set_peft_model_state_dict,
    prepare_model_for_kbit_training,
)
from peft import PeftModel

In [4]:
!git clone https://github.com/AI4Finance-Foundation/FinNLP.git
sys.path.append('/notebooks/FinNLP/')

Cloning into 'FinNLP'...
remote: Enumerating objects: 1412, done.
remote: Counting objects: 100% (471/471), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 1412 (delta 244), reused 433 (delta 224), pack-reused 941
Receiving objects: 100% (1412/1412), 4.97 MiB | 15.11 MiB/s, done.
Resolving deltas: 100% (617/617), done.


In [5]:
from finnlp.benchmarks.fpb import test_fpb
from finnlp.benchmarks.fiqa import test_fiqa , add_instructions
from finnlp.benchmarks.tfns import test_tfns
from finnlp.benchmarks.nwgi import test_nwgi

In [6]:
base_model = "NousResearch/Llama-2-13b-hf"
peft_model = "oliverwang15/FinGPT_v33_Llama2_13B_Sentiment_Instruction_LoRA_FT_8bit"

# Load the tokenizer
tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True, device_map = "auto")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [7]:
# Load the model
model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map = "auto")

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/175 [00:00<?, ?B/s]

In [8]:
# Load the Peft model
model = PeftModel.from_pretrained(model, peft_model)

adapter_config.json:   0%|          | 0.00/448 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/26.3M [00:00<?, ?B/s]

In [9]:
# Compile the model
# Note: torch.compile is used for TorchScript, make sure your platform supports it
# Comment out this line if TorchScript is not supported
model = torch.compile(model)


Process ForkProcess-8:
Process ForkProcess-1:
Process ForkProcess-6:
Process ForkProcess-9:
Process ForkProcess-3:
Process ForkProcess-7:
Process ForkProcess-5:
Traceback (most recent call last):
Process ForkProcess-4:
Traceback (most recent call last):
Process ForkProcess-2:
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/usr/lib/python3.9/multiprocessing/proce

In [10]:
# Evaluate the model
model = model.eval()

In [11]:
batch_size = 16

In [12]:
# FPB
res = test_fpb(model, tokenizer, batch_size = batch_size)

Generating train split:   0%|          | 0/4846 [00:00<?, ? examples/s]

Dataset financial_phrasebank downloaded and prepared to /root/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: L&T has also made a commitment to redeem the remaining shares by the end of 2011 .
Answer: 


Total len: 1212. Batchsize: 16. Total steps: 76


100%|██████████| 76/76 [15:18<00:00, 12.08s/it]

Acc: 0.8737623762376238. F1 macro: 0.8706682480053569. F1 micro: 0.8737623762376238. F1 weighted (BloombergGPT): 0.8739100026208315. 


In [ ]:
# NWGI, len 4047
res = test_nwgi(model, tokenizer, batch_size=batch_size)

Using custom data configuration oliverwang15--news_with_gpt_instructions-ec641c48430028ee
Reusing dataset parquet (/root/.cache/huggingface/datasets/oliverwang15___parquet/oliverwang15--news_with_gpt_instructions-ec641c48430028ee/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: In the latest trading session, Adobe Systems (ADBE) closed at $535.98, marking a +0.31% move from the previous day.
Answer: 


Total len: 4047. Batchsize: 16. Total steps: 253


 45%|████▍     | 113/253 [28:41<37:27, 16.05s/it] 